### 配置环境

In [1]:
!pip install -r requirements.txt

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 19.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
CUDA_VISIBLE_DEVICES=0 
print(torch.cuda.get_device_capability())

(8, 9)


In [4]:
print(torch.cuda.get_arch_list())

['sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']


In [5]:
print(torch.cuda.get_device_properties(torch.device('cuda')))

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4090', major=8, minor=9, total_memory=24217MB, multi_processor_count=128)


In [ ]:
#在终端运行如下指令，安装deepspeed
git clone https://github.com/microsoft/DeepSpeed/
cd DeepSpeed
rm -rf build
TORCH_CUDA_ARCH_LIST="8.9" DS_BUILD_CPU_ADAM=1 DS_BUILD_UTILS=1 pip install . \
--global-option="build_ext" --global-option="-j8" --no-cache -v \
--disable-pip-version-check 2>&1 | tee build.log

In [ ]:
# DeepSpeed ZeRO-2 模式单 GPU 训练翻译模型（T5-Small）
deepspeed --num_gpus=1 ../LLM-quickstart/deepspeed/translation/run_translation.py \
--deepspeed ../LLM-quickstart/deepspeed/config/ds_config_zero2.json \
--model_name_or_path t5-small --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro

# 运行后报错：
# ImportError: This example requires a source install from HuggingFace Transformers (see `https://huggingface.co/docs/transformers/installation#install-from-source`), but the version found is 4.37.2.

In [ ]:
# 源代码安装Transformers
pip install git+https://github.com/huggingface/transformers

In [ ]:
# 重新运行DeepSpeed ZeRO-2 模式单 GPU 训练翻译模型（T5-Small）
deepspeed --num_gpus=1 ../LLM-quickstart/deepspeed/translation/run_translation.py \
--deepspeed ../LLM-quickstart/deepspeed/config/ds_config_zero2.json \
--model_name_or_path t5-small --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro

In [ ]:
# 训练结果
[INFO|tokenization_t5_fast.py:191] 2024-02-10 21:06:01,282 >> Copy vocab file to output_dir/spiece.model
***** train metrics *****
  epoch                    =        1.0
  train_loss               =     0.9435
  train_runtime            = 0:00:47.39
  train_samples            =        500
  train_samples_per_second =      10.55
  train_steps_per_second   =      10.55

In [ ]:
# 先运行命令改变模型默认存储地址，要关注命令的运行环境，灵活更改路径
export HF_DATASETS_CACHE="/root/autodl-tmp/datasets_cache/"
export HF_HOME="/root/autodl-tmp/cache/"
export HUGGINGFACE_HUB_CACHE="/root/autodl-tmp/hub_cache/"
export TRANSFORMERS_CACHE="/root/autodl-tmp/transform_cache/"
export HF_ENDPOINT=https://hf-mirror.com

In [ ]:
# 配置网络环境
source /etc/network_turbo

In [ ]:
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-3b）,经测试60g内存是不行的，至少要90g内存。
# https://github.com/microsoft/DeepSpeed/issues/3160
deepspeed --num_gpus=1 LLM-quickstart/deepspeed/translation/run_translation.py \
--deepspeed LLM-quickstart/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-3b --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro

# 报错

In [ ]:
# 配置ds_config_zero3.json的参数，其中

# "stage3_prefetch_bucket_size": 这个参数控制每个GPU上用于预取数据的最大样本数量。
# 你已经将其设置为 32，可以尝试进一步减小这个值，例如设置为 16 或更小，以减少内存消耗。

# "stage3_param_persistence_threshold": 这个参数控制在多少步骤之后，模型参数会被持久化到硬盘上。
# 你已经将其设置为 10000，可以尝试增加这个阈值，例如设置为 20000 或更高，以减少内存消耗。

# "stage3_max_live_parameters": 这个参数控制每个GPU上最大允许的参数数量。
# 你已经将其设置为 1e4，可以尝试进一步减小这个值，例如设置为 5000 或更小，以减少内存消耗。

# "stage3_max_reuse_distance": 这个参数控制参数的最大重用距离。
# 你已经将其设置为 1e4，可以尝试进一步减小这个值，例如设置为 5000 或更小，以减少内存消耗。

# "stage3_gather_16bit_weights_on_model_save": 这个参数控制在模型保存时是否将参数转换为 16 位精度。你已经将其设置为 false，
# 这是合理的，因为不需要额外的内存来保存参数。

# 对应修改的参数如下：
stage3_prefetch_bucket_size 16
stage3_param_persistence_threshold 20000
stage3_max_live_parameters 5000
stage3_max_reuse_distance 5000
stage3_gather_16bit_weights_on_model_save false

In [ ]:
# 同时更新命令
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-3b）,经测试60g内存是不行的，至少要90g内存。
# https://github.com/microsoft/DeepSpeed/issues/3160
# bf16,为防止Exception: Current loss scale already at minimum - cannot decrease scale anymore. Exiting run.
# https://github.com/microsoft/DeepSpeedExamples/issues/418
deepspeed --num_gpus=1 LLM-quickstart/deepspeed/translation/run_translation.py \
--deepspeed LLM-quickstart/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-3b --per_device_train_batch_size 1 \
--output_dir autodl-tmp/output_dir --overwrite_output_dir --bf16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro

In [ ]:
root@autodl-container-a5904787c3-9bffb005:~# deepspeed --num_gpus=1 LLM-quickstart/deepspeed/translation/run_translation.py \
--deepspeed LLM-quickstart/deepspeed/config/ds_config_zero3.json \
--model_name_or_path t5-3b --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --bf16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[2024-02-11 02:32:52,376] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-11 02:32:53,106] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-02-11 02:32:53,106] [INFO] [runner.py:568:main] cmd = /root/miniconda3/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None LLM-quickstart/deepspeed/translation/run_translation.py --deepspeed LLM-quickstart/deepspeed/config/ds_config_zero3.json --model_name_or_path t5-3b --per_device_train_batch_size 1 --output_dir output_dir --overwrite_output_dir --bf16 --do_train --max_train_samples 500 --num_train_epochs 1 --dataset_name wmt16 --dataset_config ro-en --source_lang en --target_lang ro
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[2024-02-11 02:32:56,576] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-11 02:32:57,246] [INFO] [launch.py:138:main] 0 NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.17.1-1+cuda12.1
[2024-02-11 02:32:57,247] [INFO] [launch.py:138:main] 0 NV_LIBNCCL_DEV_PACKAGE_VERSION=2.17.1-1
[2024-02-11 02:32:57,247] [INFO] [launch.py:138:main] 0 NCCL_VERSION=2.17.1-1
[2024-02-11 02:32:57,247] [INFO] [launch.py:138:main] 0 NV_LIBNCCL_DEV_PACKAGE_NAME=libnccl-dev
[2024-02-11 02:32:57,247] [INFO] [launch.py:138:main] 0 NV_LIBNCCL_PACKAGE=libnccl2=2.17.1-1+cuda12.1
[2024-02-11 02:32:57,247] [INFO] [launch.py:138:main] 0 NV_LIBNCCL_PACKAGE_NAME=libnccl2
[2024-02-11 02:32:57,247] [INFO] [launch.py:138:main] 0 NV_LIBNCCL_PACKAGE_VERSION=2.17.1-1
[2024-02-11 02:32:57,247] [INFO] [launch.py:145:main] WORLD INFO DICT: {'localhost': [0]}
[2024-02-11 02:32:57,247] [INFO] [launch.py:151:main] nnodes=1, num_local_procs=1, node_rank=0
[2024-02-11 02:32:57,247] [INFO] [launch.py:162:main] global_rank_mapping=defaultdict(<class 'list'>, {'localhost': [0]})
[2024-02-11 02:32:57,247] [INFO] [launch.py:163:main] dist_world_size=1
[2024-02-11 02:32:57,247] [INFO] [launch.py:165:main] Setting CUDA_VISIBLE_DEVICES=0
[2024-02-11 02:32:57,247] [INFO] [launch.py:253:main] process 4518 spawned with command: ['/root/miniconda3/bin/python', '-u', 'LLM-quickstart/deepspeed/translation/run_translation.py', '--local_rank=0', '--deepspeed', 'LLM-quickstart/deepspeed/config/ds_config_zero3.json', '--model_name_or_path', 't5-3b', '--per_device_train_batch_size', '1', '--output_dir', 'output_dir', '--overwrite_output_dir', '--bf16', '--do_train', '--max_train_samples', '500', '--num_train_epochs', '1', '--dataset_name', 'wmt16', '--dataset_config', 'ro-en', '--source_lang', 'en', '--target_lang', 'ro']
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[2024-02-11 02:33:01,503] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-11 02:33:01,697] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-02-11 02:33:01,697] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
02/11/2024 02:33:01 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False
02/11/2024 02:33:01 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=LLM-quickstart/deepspeed/config/ds_config_zero3.json,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_config=None,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
include_num_input_tokens_seen=False,
include_tokens_per_second=False,
jit_mode_eval=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=0,
log_level=passive,
log_level_replica=warning,
log_on_each_node=True,
logging_dir=output_dir/runs/Feb11_02-33-00_autodl-container-a5904787c3-9bffb005,
logging_first_step=False,
logging_nan_inf_filter=True,
logging_steps=500,
logging_strategy=steps,
lr_scheduler_kwargs={},
lr_scheduler_type=linear,
max_grad_norm=1.0,
max_steps=-1,
metric_for_best_model=None,
mp_parameters=,
neftune_noise_alpha=None,
no_cuda=False,
num_train_epochs=1.0,
optim=adamw_torch,
optim_args=None,
output_dir=output_dir,
overwrite_output_dir=True,
past_index=-1,
per_device_eval_batch_size=8,
per_device_train_batch_size=1,
predict_with_generate=False,
prediction_loss_only=False,
push_to_hub=False,
push_to_hub_model_id=None,
push_to_hub_organization=None,
push_to_hub_token=<PUSH_TO_HUB_TOKEN>,
ray_scope=last,
remove_unused_columns=True,
report_to=['tensorboard'],
resume_from_checkpoint=None,
run_name=output_dir,
save_on_each_node=False,
save_only_model=False,
save_safetensors=True,
save_steps=500,
save_strategy=steps,
save_total_limit=None,
seed=42,
skip_memory_metrics=True,
sortish_sampler=False,
split_batches=False,
tf32=None,
torch_compile=False,
torch_compile_backend=None,
torch_compile_mode=None,
torchdynamo=None,
tpu_metrics_debug=False,
tpu_num_cores=None,
use_cpu=False,
use_ipex=False,
use_legacy_prediction_loop=False,
use_mps_device=False,
warmup_ratio=0.0,
warmup_steps=0,
weight_decay=0.0,
)
02/11/2024 02:33:01 - WARNING - __main__ - You're running a t5 model but didn't provide a source prefix, which is expected, e.g. with `--source_prefix 'translate English to German: ' `
Using the latest cached version of the module from /root/autodl-tmp/cache/modules/datasets_modules/datasets/wmt16/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad (last modified on Sun Feb 11 01:20:38 2024) since it couldn't be found locally at wmt16, or remotely on the Hugging Face Hub.
02/11/2024 02:33:21 - WARNING - datasets.load - Using the latest cached version of the module from /root/autodl-tmp/cache/modules/datasets_modules/datasets/wmt16/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad (last modified on Sun Feb 11 01:20:38 2024) since it couldn't be found locally at wmt16, or remotely on the Hugging Face Hub.
Loading Dataset Infos from /root/autodl-tmp/cache/modules/datasets_modules/datasets/wmt16/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad
02/11/2024 02:33:21 - INFO - datasets.info - Loading Dataset Infos from /root/autodl-tmp/cache/modules/datasets_modules/datasets/wmt16/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad
Overwrite dataset info from restored data version if exists.
02/11/2024 02:33:21 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad
02/11/2024 02:33:21 - INFO - datasets.info - Loading Dataset info from /root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad
Found cached dataset wmt16 (/root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad)
02/11/2024 02:33:21 - INFO - datasets.builder - Found cached dataset wmt16 (/root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad)
Loading Dataset info from /root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad
02/11/2024 02:33:21 - INFO - datasets.info - Loading Dataset info from /root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad
[INFO|configuration_utils.py:729] 2024-02-11 02:33:32,049 >> loading configuration file config.json from cache at /root/autodl-tmp/transform_cache/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/config.json
[INFO|configuration_utils.py:792] 2024-02-11 02:33:32,060 >> Model config T5Config {
  "_name_or_path": "t5-3b",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 32,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to French: "
    },
    "translation_en_to_ro": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Romanian: "
    }
  },
  "transformers_version": "4.38.0.dev0",
  "use_cache": true,
  "vocab_size": 32128
}

[INFO|tokenization_auto.py:607] 2024-02-11 02:33:42,312 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:729] 2024-02-11 02:33:52,724 >> loading configuration file config.json from cache at /root/autodl-tmp/transform_cache/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/config.json
[INFO|configuration_utils.py:792] 2024-02-11 02:33:52,727 >> Model config T5Config {
  "_name_or_path": "t5-3b",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 32,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to French: "
    },
    "translation_en_to_ro": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Romanian: "
    }
  },
  "transformers_version": "4.38.0.dev0",
  "use_cache": true,
  "vocab_size": 32128
}

[INFO|tokenization_utils_base.py:2029] 2024-02-11 02:34:13,365 >> loading file spiece.model from cache at /root/autodl-tmp/transform_cache/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/spiece.model
[INFO|tokenization_utils_base.py:2029] 2024-02-11 02:34:13,365 >> loading file tokenizer.json from cache at /root/autodl-tmp/transform_cache/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/tokenizer.json
[INFO|tokenization_utils_base.py:2029] 2024-02-11 02:34:13,366 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2029] 2024-02-11 02:34:13,366 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2029] 2024-02-11 02:34:13,366 >> loading file tokenizer_config.json from cache at None
[INFO|configuration_utils.py:729] 2024-02-11 02:34:13,366 >> loading configuration file config.json from cache at /root/autodl-tmp/transform_cache/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/config.json
[INFO|configuration_utils.py:792] 2024-02-11 02:34:13,380 >> Model config T5Config {
  "_name_or_path": "t5-3b",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 32,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to French: "
    },
    "translation_en_to_ro": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Romanian: "
    }
  },
  "transformers_version": "4.38.0.dev0",
  "use_cache": true,
  "vocab_size": 32128
}

/root/miniconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-3b automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
[INFO|modeling_utils.py:3259] 2024-02-11 02:34:13,717 >> loading weights file model.safetensors from cache at /root/autodl-tmp/transform_cache/models--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0/model.safetensors
[INFO|modeling_utils.py:3365] 2024-02-11 02:34:13,743 >> Detected DeepSpeed ZeRO-3: activating zero.init() for this model
[INFO|configuration_utils.py:840] 2024-02-11 02:34:13,753 >> Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}

[2024-02-11 02:34:18,735] [INFO] [partition_parameters.py:343:__exit__] finished initializing model - num_params = 510, num_elems = 2.88B
[INFO|modeling_utils.py:3992] 2024-02-11 02:34:21,579 >> All model checkpoint weights were used when initializing T5ForConditionalGeneration.

[INFO|modeling_utils.py:4000] 2024-02-11 02:34:21,579 >> All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at t5-3b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.
[INFO|modeling_utils.py:3546] 2024-02-11 02:34:31,781 >> Generation config file not found, using a generation config created from the model config.
[INFO|modeling_utils.py:1875] 2024-02-11 02:34:31,796 >> You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32100. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
Loading cached processed dataset at /root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad/cache-d11b78484a2e05f1.arrow
02/11/2024 02:34:32 - INFO - datasets.arrow_dataset - Loading cached processed dataset at /root/autodl-tmp/datasets_cache/wmt16/ro-en/1.0.0/f5dc442f4d1c2cc487cd2d5591af56c03a5f03bb98a3bb92151d015c8c9cb7ad/cache-d11b78484a2e05f1.arrow
02/11/2024 02:36:37 - WARNING - evaluate.loading - Using the latest cached version of the module from /root/autodl-tmp/cache/modules/evaluate_modules/metrics/evaluate-metric--sacrebleu/28676bf65b4f88b276df566e48e603732d0b4afd237603ebdf92acaacf5be99b (last modified on Sun Feb 11 01:33:07 2024) since it couldn't be found locally at evaluate-metric--sacrebleu, or remotely on the Hugging Face Hub.
[INFO|trainer.py:586] 2024-02-11 02:36:37,161 >> Using auto half precision backend
[2024-02-11 02:36:37,360] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.13.2+25a02047, git-hash=25a02047, git-branch=master
[2024-02-11 02:36:37,388] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Adam Optimizer #0 is created with AVX512 arithmetic capability.
Config: alpha=0.000050, betas=(0.900000, 0.999000), weight_decay=0.000000, adam_w=1
[2024-02-11 02:36:39,147] [INFO] [logging.py:96:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adamw as basic optimizer
[2024-02-11 02:36:39,147] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2024-02-11 02:36:39,215] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = DeepSpeedCPUAdam
[2024-02-11 02:36:39,215] [INFO] [utils.py:56:is_zero_supported_optimizer] Checking ZeRO support for optimizer=DeepSpeedCPUAdam type=<class 'deepspeed.ops.adam.cpu_adam.DeepSpeedCPUAdam'>
[2024-02-11 02:36:39,215] [INFO] [logging.py:96:log_dist] [Rank 0] Creating fp16 ZeRO stage 3 optimizer, MiCS is enabled False, Hierarchical params gather False
[2024-02-11 02:36:39,215] [INFO] [logging.py:96:log_dist] [Rank 0] Creating torch.bfloat16 ZeRO stage 3 optimizer
[2024-02-11 02:36:39,347] [INFO] [utils.py:800:see_memory_usage] Stage 3 initialize beginning
[2024-02-11 02:36:39,348] [INFO] [utils.py:801:see_memory_usage] MA 0.06 GB         Max_MA 0.18 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:39,348] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 46.58 GB, percent = 4.6%
[2024-02-11 02:36:39,355] [INFO] [stage3.py:130:__init__] Reduce bucket size 1048576
[2024-02-11 02:36:39,355] [INFO] [stage3.py:131:__init__] Prefetch bucket size 16
[2024-02-11 02:36:39,474] [INFO] [utils.py:800:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2024-02-11 02:36:39,475] [INFO] [utils.py:801:see_memory_usage] MA 0.06 GB         Max_MA 0.06 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:39,475] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 46.58 GB, percent = 4.6%
Parameter Offload: Total persistent parameters: 126976 in 124 params
[2024-02-11 02:36:39,700] [INFO] [utils.py:800:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2024-02-11 02:36:39,701] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.06 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:39,702] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 46.58 GB, percent = 4.6%
[2024-02-11 02:36:39,828] [INFO] [utils.py:800:see_memory_usage] Before creating fp16 partitions
[2024-02-11 02:36:39,829] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:39,829] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 46.58 GB, percent = 4.6%
[2024-02-11 02:36:43,561] [INFO] [utils.py:800:see_memory_usage] After creating fp16 partitions: 3
[2024-02-11 02:36:43,562] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:43,562] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 54.78 GB, percent = 5.4%
[2024-02-11 02:36:43,666] [INFO] [utils.py:800:see_memory_usage] Before creating fp32 partitions
[2024-02-11 02:36:43,666] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:43,667] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 54.78 GB, percent = 5.4%
[2024-02-11 02:36:44,853] [INFO] [utils.py:800:see_memory_usage] After creating fp32 partitions
[2024-02-11 02:36:44,854] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:44,854] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 67.23 GB, percent = 6.7%
[2024-02-11 02:36:44,955] [INFO] [utils.py:800:see_memory_usage] Before initializing optimizer states
[2024-02-11 02:36:44,955] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:44,955] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 67.23 GB, percent = 6.7%
[2024-02-11 02:36:53,404] [INFO] [utils.py:800:see_memory_usage] After initializing optimizer states
[2024-02-11 02:36:53,405] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.06 GB         Max_CA 0 GB 
[2024-02-11 02:36:53,405] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 100.72 GB, percent = 10.0%
[2024-02-11 02:36:53,406] [INFO] [stage3.py:487:_setup_for_real_optimizer] optimizer state initialized
[2024-02-11 02:36:57,802] [INFO] [utils.py:800:see_memory_usage] After initializing ZeRO optimizer
[2024-02-11 02:36:57,803] [INFO] [utils.py:801:see_memory_usage] MA 0.0 GB         Max_MA 0.12 GB         CA 0.19 GB         Max_CA 0 GB 
[2024-02-11 02:36:57,803] [INFO] [utils.py:808:see_memory_usage] CPU Virtual Memory:  used = 108.91 GB, percent = 10.8%
[2024-02-11 02:36:57,803] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Final Optimizer = adamw
[2024-02-11 02:36:57,803] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed using configured LR scheduler = WarmupLR
[2024-02-11 02:36:57,803] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed LR Scheduler = <deepspeed.runtime.lr_schedules.WarmupLR object at 0x7f31782a3130>
[2024-02-11 02:36:57,803] [INFO] [logging.py:96:log_dist] [Rank 0] step=0, skipped=0, lr=[5e-05], mom=[[0.9, 0.999]]
[2024-02-11 02:36:57,805] [INFO] [config.py:987:print] DeepSpeedEngine configuration:
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   activation_checkpointing_config  {
    "partition_activations": false, 
    "contiguous_memory_optimization": false, 
    "cpu_checkpointing": false, 
    "number_checkpoints": null, 
    "synchronize_checkpoint_boundary": false, 
    "profile": false
}
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   aio_config ................... {'block_size': 1048576, 'queue_depth': 8, 'thread_count': 1, 'single_submit': False, 'overlap_events': True}
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   amp_enabled .................. False
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   amp_params ................... False
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   autotuning_config ............ {
    "enabled": false, 
    "start_step": null, 
    "end_step": null, 
    "metric_path": null, 
    "arg_mappings": null, 
    "metric": "throughput", 
    "model_info": null, 
    "results_dir": "autotuning_results", 
    "exps_dir": "autotuning_exps", 
    "overwrite": true, 
    "fast": true, 
    "start_profile_step": 3, 
    "end_profile_step": 5, 
    "tuner_type": "gridsearch", 
    "tuner_early_stopping": 5, 
    "tuner_num_trials": 50, 
    "model_info_path": null, 
    "mp_size": 1, 
    "max_train_batch_size": null, 
    "min_train_batch_size": 1, 
    "max_train_micro_batch_size_per_gpu": 1.024000e+03, 
    "min_train_micro_batch_size_per_gpu": 1, 
    "num_tuning_micro_batch_sizes": 3
}
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   bfloat16_enabled ............. True
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   checkpoint_parallel_write_pipeline  False
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   checkpoint_tag_validation_enabled  True
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   checkpoint_tag_validation_fail  False
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   comms_config ................. <deepspeed.comm.config.DeepSpeedCommsConfig object at 0x7f31a01a9bd0>
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   communication_data_type ...... None
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   compile_config ............... enabled=False backend='inductor' kwargs={}
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   compression_config ........... {'weight_quantization': {'shared_parameters': {'enabled': False, 'quantizer_kernel': False, 'schedule_offset': 0, 'quantize_groups': 1, 'quantize_verbose': False, 'quantization_type': 'symmetric', 'quantize_weight_in_forward': False, 'rounding': 'nearest', 'fp16_mixed_quantize': False, 'quantize_change_ratio': 0.001}, 'different_groups': {}}, 'activation_quantization': {'shared_parameters': {'enabled': False, 'quantization_type': 'symmetric', 'range_calibration': 'dynamic', 'schedule_offset': 1000}, 'different_groups': {}}, 'sparse_pruning': {'shared_parameters': {'enabled': False, 'method': 'l1', 'schedule_offset': 1000}, 'different_groups': {}}, 'row_pruning': {'shared_parameters': {'enabled': False, 'method': 'l1', 'schedule_offset': 1000}, 'different_groups': {}}, 'head_pruning': {'shared_parameters': {'enabled': False, 'method': 'topk', 'schedule_offset': 1000}, 'different_groups': {}}, 'channel_pruning': {'shared_parameters': {'enabled': False, 'method': 'l1', 'schedule_offset': 1000}, 'different_groups': {}}, 'layer_reduction': {'enabled': False}}
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   curriculum_enabled_legacy .... False
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   curriculum_params_legacy ..... False
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   data_efficiency_config ....... {'enabled': False, 'seed': 1234, 'data_sampling': {'enabled': False, 'num_epochs': 1000, 'num_workers': 0, 'curriculum_learning': {'enabled': False}}, 'data_routing': {'enabled': False, 'random_ltd': {'enabled': False, 'layer_token_lr_schedule': {'enabled': False}}}}
[2024-02-11 02:36:57,806] [INFO] [config.py:991:print]   data_efficiency_enabled ...... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   dataloader_drop_last ......... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   disable_allgather ............ False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   dump_state ................... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   dynamic_loss_scale_args ...... None
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_enabled ........... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_gas_boundary_resolution  1
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_layer_name ........ bert.encoder.layer
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_layer_num ......... 0
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_max_iter .......... 100
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_stability ......... 1e-06
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_tol ............... 0.01
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   eigenvalue_verbose ........... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   elasticity_enabled ........... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   flops_profiler_config ........ {
    "enabled": false, 
    "recompute_fwd_factor": 0.0, 
    "profile_step": 1, 
    "module_depth": -1, 
    "top_modules": 1, 
    "detailed": true, 
    "output_file": null
}
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   fp16_auto_cast ............... None
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   fp16_enabled ................. False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   fp16_master_weights_and_gradients  False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   global_rank .................. 0
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   grad_accum_dtype ............. None
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   gradient_accumulation_steps .. 1
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   gradient_clipping ............ 1.0
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   gradient_predivide_factor .... 1.0
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   graph_harvesting ............. False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   hybrid_engine ................ enabled=False max_out_tokens=512 inference_tp_size=1 release_inference_cache=False pin_parameters=True tp_gather_partition_size=8
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   initial_dynamic_scale ........ 1
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   load_universal_checkpoint .... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   loss_scale ................... 1.0
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   memory_breakdown ............. False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   mics_hierarchial_params_gather  False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   mics_shard_size .............. -1
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   monitor_config ............... tensorboard=TensorBoardConfig(enabled=False, output_path='', job_name='DeepSpeedJobName') wandb=WandbConfig(enabled=False, group=None, team=None, project='deepspeed') csv_monitor=CSVConfig(enabled=False, output_path='', job_name='DeepSpeedJobName') enabled=False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   nebula_config ................ {
    "enabled": false, 
    "persistent_storage_path": null, 
    "persistent_time_interval": 100, 
    "num_of_version_in_retention": 2, 
    "enable_nebula_load": true, 
    "load_path": null
}
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   optimizer_legacy_fusion ...... False
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   optimizer_name ............... adamw
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   optimizer_params ............. {'lr': 5e-05, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0.0}
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   pipeline ..................... {'stages': 'auto', 'partition': 'best', 'seed_layers': False, 'activation_checkpoint_interval': 0, 'pipe_partitioned': True, 'grad_partitioned': True}
[2024-02-11 02:36:57,807] [INFO] [config.py:991:print]   pld_enabled .................. False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   pld_params ................... False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   prescale_gradients ........... False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   scheduler_name ............... WarmupLR
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   scheduler_params ............. {'warmup_min_lr': 0, 'warmup_max_lr': 5e-05, 'warmup_num_steps': 0}
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   seq_parallel_communication_data_type  torch.float32
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   sparse_attention ............. None
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   sparse_gradients_enabled ..... False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   steps_per_print .............. inf
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   train_batch_size ............. 1
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   train_micro_batch_size_per_gpu  1
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   use_data_before_expert_parallel_  False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   use_node_local_storage ....... False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   wall_clock_breakdown ......... False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   weight_quantization_config ... None
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   world_size ................... 1
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   zero_allow_untested_optimizer  False
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   zero_config .................. stage=3 contiguous_gradients=True reduce_scatter=True reduce_bucket_size=1048576 use_multi_rank_bucket_allreduce=True allgather_partitions=True allgather_bucket_size=500,000,000 overlap_comm=True load_from_fp32_weights=True elastic_checkpoint=False offload_param=DeepSpeedZeroOffloadParamConfig(device='cpu', nvme_path=None, buffer_count=5, buffer_size=100,000,000, max_in_cpu=1,000,000,000, pin_memory=True) offload_optimizer=DeepSpeedZeroOffloadOptimizerConfig(device='cpu', nvme_path=None, buffer_count=4, pin_memory=True, pipeline=False, pipeline_read=False, pipeline_write=False, fast_init=False, ratio=1.0) sub_group_size=1000000000 cpu_offload_param=None cpu_offload_use_pin_memory=None cpu_offload=None prefetch_bucket_size=16 param_persistence_threshold=20000 model_persistence_threshold=sys.maxsize max_live_parameters=5000 max_reuse_distance=5000 gather_16bit_weights_on_model_save=False stage3_gather_fp16_weights_on_model_save=False ignore_unused_parameters=True legacy_stage1=False round_robin_gradients=False zero_hpz_partition_size=1 zero_quantized_weights=False zero_quantized_nontrainable_weights=False zero_quantized_gradients=False mics_shard_size=-1 mics_hierarchical_params_gather=False memory_efficient_linear=True pipeline_loading_checkpoint=False override_module_apply=True
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   zero_enabled ................. True
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   zero_force_ds_cpu_optimizer .. True
[2024-02-11 02:36:57,808] [INFO] [config.py:991:print]   zero_optimization_stage ...... 3
[2024-02-11 02:36:57,808] [INFO] [config.py:977:print_user_config]   json = {
    "fp16": {
        "enabled": false, 
        "loss_scale": 0, 
        "loss_scale_window": 1000, 
        "initial_scale_power": 16, 
        "hysteresis": 2, 
        "min_loss_scale": 1
    }, 
    "bf16": {
        "enabled": true
    }, 
    "optimizer": {
        "type": "AdamW", 
        "params": {
            "lr": 5e-05, 
            "betas": [0.9, 0.999], 
            "eps": 1e-08, 
            "weight_decay": 0.0
        }
    }, 
    "scheduler": {
        "type": "WarmupLR", 
        "params": {
            "warmup_min_lr": 0, 
            "warmup_max_lr": 5e-05, 
            "warmup_num_steps": 0
        }
    }, 
    "zero_optimization": {
        "stage": 3, 
        "offload_optimizer": {
            "device": "cpu", 
            "pin_memory": true
        }, 
        "offload_param": {
            "device": "cpu", 
            "pin_memory": true
        }, 
        "overlap_comm": true, 
        "contiguous_gradients": true, 
        "sub_group_size": 1.000000e+09, 
        "reduce_bucket_size": 1.048576e+06, 
        "stage3_prefetch_bucket_size": 16, 
        "stage3_param_persistence_threshold": 2.000000e+04, 
        "stage3_max_live_parameters": 5.000000e+03, 
        "stage3_max_reuse_distance": 5.000000e+03, 
        "stage3_gather_16bit_weights_on_model_save": false
    }, 
    "gradient_accumulation_steps": 1, 
    "gradient_clipping": 1.0, 
    "steps_per_print": inf, 
    "train_batch_size": 1, 
    "train_micro_batch_size_per_gpu": 1, 
    "wall_clock_breakdown": false
}
[INFO|trainer.py:1747] 2024-02-11 02:36:57,808 >> ***** Running training *****
[INFO|trainer.py:1748] 2024-02-11 02:36:57,808 >>   Num examples = 500
[INFO|trainer.py:1749] 2024-02-11 02:36:57,809 >>   Num Epochs = 1
[INFO|trainer.py:1750] 2024-02-11 02:36:57,809 >>   Instantaneous batch size per device = 1
[INFO|trainer.py:1753] 2024-02-11 02:36:57,809 >>   Total train batch size (w. parallel, distributed & accumulation) = 1
[INFO|trainer.py:1754] 2024-02-11 02:36:57,809 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1755] 2024-02-11 02:36:57,809 >>   Total optimization steps = 500
[INFO|trainer.py:1756] 2024-02-11 02:36:57,811 >>   Number of trainable parameters = 2,851,569,664
  0%|                                                                                                                 | 0/500 [00:00<?, ?it/s]/root/miniconda3/lib/python3.10/site-packages/deepspeed/runtime/zero/stage3.py:1403: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
    total_norm_cuda = get_accelerator().FloatTensor([float(total_norm)])
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [47:32<00:00,  5.71s/it]
[WARNING|trainer.py:2920] 2024-02-11 15:36:37,574 >>  stage3_gather_16bit_weights_on_model_save=false. Saving the full checkpoint instead, use zero_to_fp32.py to recover weights
[INFO|trainer.py:2981] 2024-02-11 15:36:37,574 >> Saving model checkpoint to autodl-tmp/output_dir
[INFO|configuration_utils.py:473] 2024-02-11 15:36:37,575 >> Configuration saved in autodl-tmp/output_dir/config.json
[INFO|configuration_utils.py:608] 2024-02-11 15:36:37,576 >> Configuration saved in autodl-tmp/output_dir/generation_config.json
[INFO|modeling_utils.py:2454] 2024-02-11 15:36:37,576 >> Model weights saved in autodl-tmp/output_dir/model.safetensors
[INFO|tokenization_utils_base.py:2435] 2024-02-11 15:36:37,578 >> tokenizer config file saved in autodl-tmp/output_dir/tokenizer_config.json
[INFO|tokenization_utils_base.py:2444] 2024-02-11 15:36:37,578 >> Special tokens file saved in autodl-tmp/output_dir/special_tokens_map.json
[INFO|tokenization_t5_fast.py:191] 2024-02-11 15:36:37,579 >> Copy vocab file to autodl-tmp/output_dir/spiece.model
[2024-02-11 15:36:37,609] [INFO] [logging.py:96:log_dist] [Rank 0] [Torch] Checkpoint global_step500 is about to be saved!
/root/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1879: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
[2024-02-11 15:36:37,627] [INFO] [logging.py:96:log_dist] [Rank 0] Saving model checkpoint: autodl-tmp/output_dir/global_step500/zero_pp_rank_0_mp_rank_00_model_states.pt
[2024-02-11 15:36:37,627] [INFO] [torch_checkpoint_engine.py:21:save] [Torch] Saving autodl-tmp/output_dir/global_step500/zero_pp_rank_0_mp_rank_00_model_states.pt...
[2024-02-11 15:36:37,650] [INFO] [torch_checkpoint_engine.py:23:save] [Torch] Saved autodl-tmp/output_dir/global_step500/zero_pp_rank_0_mp_rank_00_model_states.pt.
[2024-02-11 15:36:37,651] [INFO] [torch_checkpoint_engine.py:21:save] [Torch] Saving autodl-tmp/output_dir/global_step500/bf16_zero_pp_rank_0_mp_rank_00_optim_states.pt...
[2024-02-11 15:37:17,127] [INFO] [torch_checkpoint_engine.py:23:save] [Torch] Saved autodl-tmp/output_dir/global_step500/bf16_zero_pp_rank_0_mp_rank_00_optim_states.pt.
[2024-02-11 15:37:17,128] [INFO] [engine.py:3487:_save_zero_checkpoint] zero checkpoint saved autodl-tmp/output_dir/global_step500/bf16_zero_pp_rank_0_mp_rank_00_optim_states.pt
[2024-02-11 15:37:17,138] [INFO] [torch_checkpoint_engine.py:33:commit] [Torch] Checkpoint global_step500 is ready now!
***** train metrics *****
  epoch                    =        1.0
  train_loss               =      0.558
  train_runtime            = 0:47:32.65
  train_samples            =        500
  train_samples_per_second =      0.175
  train_steps_per_second   =      0.175
[INFO|modelcard.py:452] 2024-02-11 15:37:17,935 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Translation', 'type': 'translation'}, 'dataset': {'name': 'wmt16 ro-en', 'type': 'wmt16', 'config': 'ro-en', 'split': 'train', 'args': 'ro-en'}}
[2024-02-11 15:37:29,404] [INFO] [launch.py:348:main] Process 1169 exits successfully.

In [ ]:
内存占用
Every 1.0s: nvidia-smi                                                          autodl-container-a5904787c3-9bffb005: Sun Feb 11 02:40:19 2024

Sun Feb 11 02:40:19 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:32:00.0 Off |                  Off |
|  0%   44C    P2              71W / 450W |   1911MiB / 24564MiB |     41%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
+---------------------------------------------------------------------------------------+